In [1]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CIPJ'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [2]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CIPJ.xlsx')
time.sleep(random.randint(60, 80))

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2023-08-31_15-09_CIPJ_relatorio_almox.xlsx')
time.sleep(random.randint(60, 80))

logger.info(f'tratamdp dados da planilha de levantameto fisico {n_campi}')
time.sleep(random.randint(60, 80))

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]
# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

print(levantamento.info())
print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código', 'Retificação'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})
# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
time.sleep(random.randint(60, 80))
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')
time.sleep(random.randint(60, 80))

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
display(suap_cross)
display(levantamento_cross)

logger.info(f'consolidando dados')
time.sleep(random.randint(60, 80))

logger.info(f'exportando planilha de consolidação')
time.sleep(random.randint(60, 80))

consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

logger.info(f'consolidação campus {n_campi} finalizada')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Carimbo de data/hora                  497 non-null    datetime64[ns]
 1   Endereço de e-mail                    497 non-null    object        
 2   Retificação                           10 non-null     object        
 3   Código                                497 non-null    object        
 4   Nome                                  497 non-null    object        
 5   Quantidade                            497 non-null    int64         
 6   Campus                                497 non-null    object        
 7   Localização                           305 non-null    object        
 8   Valor unitário                        497 non-null    float64       
 9   Sem identificação - SI                0 non-null      float64       
 10  Ma

,codigo,descricao_suap,und,qtd_suap,ed
0,30.07.1222,"BISCOITO, APRESENTAÇÃO QUADRADO, CLASSIFICAÇÃO...",UN,5,07 - GENEROS DE ALIMENTACAO
1,30.07.1316,"BISCOITO, APRESENTAÇÃO RETANGULAR, SABOR MAIZE...",PCT,5,07 - GENEROS DE ALIMENTACAO
2,30.07.1431,"CAFÉ, APRESENTAÇÃO TORRADO MOÍDO, INTENSIDADE ...",PCT,240,07 - GENEROS DE ALIMENTACAO
3,30.07.1462,Açucar cristal 1KG,UN,170,07 - GENEROS DE ALIMENTACAO
4,30.07.219,ADOÇANTE ARTIFICIAL LIQUIDO PARA CONSUMO. EMBA...,UN,12,07 - GENEROS DE ALIMENTACAO
...,...,...,...,...,...
506,30.42.892,"JOGO CHAVE, MATERIAL AÇO CROMO VANÁDIO, TIPO E...",UN,7,42 - FERRAMENTAS
507,30.42.974,"CHAVE FENDA, MATERIAL HASTE AÇO VANÁDIO TREFIL...",UN,12,42 - FERRAMENTAS
508,30.42.975,"CHAVE DE FENDA, MATERIAL AÇO CROMO VANÁDIO, CA...",UN,6,42 - FERRAMENTAS
509,30.42.976,"CHAVE DE FENDA, MATERIAL HASTE AÇO CROMO VANÁD...",UN,8,42 - FERRAMENTAS


,codigo,descricao_lev,qtd_lev,servidor
0,30.24.3204,"MASSA CORRIDA, MÉTODO APLICAÇÃO COM ESPÁTULA E...",2,claudeandela.cavalcanti@reitoria.ifpe.edu.br
1,30.24.1405,TINTA ACRILICA PARA PISOS NA COR AMARELA LATA 18L,3,luiz.araujo@ipojuca.ifpe.edu.br
3,30.16.422,PAPEL A4 ALCALINO 297 X 210MM 75 G/M2 - RESMA ...,936,marcellosouza@ipojuca.ifpe.edu.br
4,30.24.3202,"TINTA ACRILICA, COR VERDE ESMERALDA, 18 LTS",8,luiz.araujo@ipojuca.ifpe.edu.br
5,30.22.302,"PAPEL HIGIÊNICO, FOLHA SIMPLES, TAMANHO 10CM X...",9,marcellosouza@ipojuca.ifpe.edu.br
...,...,...,...,...
492,30.26.3069,"Tomada, modelo plugue, fêmea, corrente nominal...",2,cmpa@ipojuca.ifpe.edu.br
493,30.26.1206,"Conector capa HD15, para montagem de cabo VGA",7,cmpa@ipojuca.ifpe.edu.br
494,30.22.371,"Espanador, cabo de madeira, medindo 35 cm.",22,cmpa@ipojuca.ifpe.edu.br
495,30.22.559,LIXEIRA COM PEDAL PLASTICA. COR: BRANCA. MARCA...,12,cmpa@ipojuca.ifpe.edu.br


In [3]:
logger.info(f'montando planilha de ajustes ')
time.sleep(random.randint(60, 80))

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')
time.sleep(random.randint(60, 80))

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.07.1222,"BISCOITO, APRESENTAÇÃO QUADRADO, CLASSIFICAÇÃO...",UN,5,07 - GENEROS DE ALIMENTACAO,"BISCOITO, APRESENTAÇÃO QUADRADO, CLASSIFICAÇÃO...",5.0,luiz.araujo@ipojuca.ifpe.edu.br,both,0.0
1,30.07.1316,"BISCOITO, APRESENTAÇÃO RETANGULAR, SABOR MAIZE...",PCT,5,07 - GENEROS DE ALIMENTACAO,"BISCOITO, APRESENTAÇÃO RETANGULAR, SABOR MAIZE...",5.0,luiz.araujo@ipojuca.ifpe.edu.br,both,0.0
2,30.07.1431,"CAFÉ, APRESENTAÇÃO TORRADO MOÍDO, INTENSIDADE ...",PCT,240,07 - GENEROS DE ALIMENTACAO,"CAFÉ, APRESENTAÇÃO TORRADO MOÍDO, INTENSIDADE ...",12.0,luiz.araujo@ipojuca.ifpe.edu.br,both,228.0
3,30.07.1462,Açucar cristal 1KG,UN,170,07 - GENEROS DE ALIMENTACAO,Açucar cristal 1KG,170.0,luiz.araujo@ipojuca.ifpe.edu.br,both,0.0
4,30.07.219,ADOÇANTE ARTIFICIAL LIQUIDO PARA CONSUMO. EMBA...,UN,12,07 - GENEROS DE ALIMENTACAO,ADOÇANTE ARTIFICIAL LIQUIDO PARA CONSUMO. EMBA...,12.0,luiz.araujo@ipojuca.ifpe.edu.br,both,0.0
...,...,...,...,...,...,...,...,...,...,...
514,30.42.892,"JOGO CHAVE, MATERIAL AÇO CROMO VANÁDIO, TIPO E...",UN,7,42 - FERRAMENTAS,"JOGO CHAVE, MATERIAL AÇO CROMO VANÁDIO, TIPO E...",7.0,luiz.araujo@ipojuca.ifpe.edu.br,both,0.0
515,30.42.974,"CHAVE FENDA, MATERIAL HASTE AÇO VANÁDIO TREFIL...",UN,12,42 - FERRAMENTAS,"CHAVE FENDA, MATERIAL HASTE AÇO VANÁDIO TREFIL...",12.0,luiz.araujo@ipojuca.ifpe.edu.br,both,0.0
516,30.42.975,"CHAVE DE FENDA, MATERIAL AÇO CROMO VANÁDIO, CA...",UN,6,42 - FERRAMENTAS,"CHAVE DE FENDA, MATERIAL AÇO CROMO VANÁDIO, CA...",6.0,luiz.araujo@ipojuca.ifpe.edu.br,both,0.0
517,30.42.976,"CHAVE DE FENDA, MATERIAL HASTE AÇO CROMO VANÁD...",UN,8,42 - FERRAMENTAS,"CHAVE DE FENDA, MATERIAL HASTE AÇO CROMO VANÁD...",8.0,luiz.araujo@ipojuca.ifpe.edu.br,both,0.0


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.07.1222,07 - GENEROS DE ALIMENTACAO,"BISCOITO, APRESENTAÇÃO QUADRADO, CLASSIFICAÇÃO...",5,5.0,0.0,adequado
1,30.07.1316,07 - GENEROS DE ALIMENTACAO,"BISCOITO, APRESENTAÇÃO RETANGULAR, SABOR MAIZE...",5,5.0,0.0,adequado
2,30.07.1431,07 - GENEROS DE ALIMENTACAO,"CAFÉ, APRESENTAÇÃO TORRADO MOÍDO, INTENSIDADE ...",240,12.0,228.0,pedido no suap
3,30.07.1462,07 - GENEROS DE ALIMENTACAO,Açucar cristal 1KG,170,170.0,0.0,adequado
4,30.07.219,07 - GENEROS DE ALIMENTACAO,ADOÇANTE ARTIFICIAL LIQUIDO PARA CONSUMO. EMBA...,12,12.0,0.0,adequado
...,...,...,...,...,...,...,...
514,30.42.892,42 - FERRAMENTAS,"JOGO CHAVE, MATERIAL AÇO CROMO VANÁDIO, TIPO E...",7,7.0,0.0,adequado
515,30.42.974,42 - FERRAMENTAS,"CHAVE FENDA, MATERIAL HASTE AÇO VANÁDIO TREFIL...",12,12.0,0.0,adequado
516,30.42.975,42 - FERRAMENTAS,"CHAVE DE FENDA, MATERIAL AÇO CROMO VANÁDIO, CA...",6,6.0,0.0,adequado
517,30.42.976,42 - FERRAMENTAS,"CHAVE DE FENDA, MATERIAL HASTE AÇO CROMO VANÁD...",8,8.0,0.0,adequado
